In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [12]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.regression import LinearRegression

def extract_features(r): #line별
    name=[int(subway_dict[r[0]])]
    number=[float(r[i]) for i in range(1,len(r)-1)]
    return name+number

def extract_label(r):
    return int(r[len(r)-1]) # int는 소문자로 지정해야 함

In [22]:
raw_data=sc.textFile("/home/csxion/Desktop/project/data/1601/1601_07_standard.csv")
records=raw_data.map(lambda line:line.split(","))
subway_dict=records.map(lambda r:r[0]).distinct().zipWithIndex().collectAsMap() # distinct 꼭 사용할 것

labels=records.map(lambda r:extract_label(r))
features=records.map(lambda r:Vectors.dense(np.array(extract_features(r))))
labels_and_features=labels.zip(features)
df=spark.createDataFrame(labels_and_features, ("label : int", "features : "))

In [29]:
df=spark.createDataFrame(records.map(lambda line:(extract_label(line), Vectors.dense(np.array(extract_features(line)))))).toDF("label", "features")

In [31]:
df.take(10)

[Row(label=78, features=DenseVector([39.0, 5.0, 0.4889, 0.492, 0.3774, 0.9008, 0.9466, 0.9223, 0.9327])),
 Row(label=183, features=DenseVector([39.0, 6.0, 0.4889, 0.492, 0.1452, 0.9008, 0.9466, 0.9223, 0.9327])),
 Row(label=143, features=DenseVector([39.0, 7.0, 0.4963, 0.492, 0.2302, 0.9008, 0.9466, 0.9223, 0.9327])),
 Row(label=261, features=DenseVector([39.0, 8.0, 0.4814, 0.492, 0.3774, 0.9008, 0.9466, 0.9223, 0.9327])),
 Row(label=286, features=DenseVector([39.0, 9.0, 0.5038, 0.492, 0.102, 0.9008, 0.9466, 0.9223, 0.9327])),
 Row(label=316, features=DenseVector([39.0, 10.0, 0.5781, 0.492, 0.1222, 0.9008, 0.9466, 0.9223, 0.9327])),
 Row(label=393, features=DenseVector([39.0, 11.0, 0.697, 0.492, 0.2638, 0.9008, 0.9466, 0.9223, 0.9327])),
 Row(label=424, features=DenseVector([39.0, 12.0, 0.7818, 0.492, 0.2997, 0.9008, 0.9466, 0.9223, 0.9327])),
 Row(label=452, features=DenseVector([39.0, 13.0, 0.8184, 0.492, 0.2302, 0.9008, 0.9466, 0.9223, 0.9327])),
 Row(label=467, features=DenseVector

In [33]:
lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
lrModel = lr.fit(df)

Py4JJavaError: An error occurred while calling o1136.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 53.0 failed 1 times, most recent failure: Lost task 1.0 in stage 53.0 (TID 72, localhost): scala.MatchError: [59,1.0,[0.0,6.0,0.488862391,0.492025331,0.230222403,0.578656794,0.487342132,0.518121538,0.509604053]] (of class org.apache.spark.sql.catalyst.expressions.GenericRowWithSchema)
	at org.apache.spark.ml.regression.LinearRegression$$anonfun$5.apply(LinearRegression.scala:200)
	at org.apache.spark.ml.regression.LinearRegression$$anonfun$5.apply(LinearRegression.scala:200)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:409)
	at org.apache.spark.storage.memory.MemoryStore.putIteratorAsValues(MemoryStore.scala:214)
	at org.apache.spark.storage.BlockManager$$anonfun$doPutIterator$1.apply(BlockManager.scala:935)
	at org.apache.spark.storage.BlockManager$$anonfun$doPutIterator$1.apply(BlockManager.scala:926)
	at org.apache.spark.storage

In [ ]:
# Print the coefficients and intercept for linear regression
print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))

# Summarize the model over the training set and print out some metrics
trainingSummary = lrModel.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

In [35]:
df

DataFrame[label: bigint, features: vector]

In [ ]:
df